##Thiết lập môi trường

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/"

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
!nvidia-smi

Fri Jul  2 12:38:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    32W /  70W |   2506MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install -q pytorch-lightning==1.2.8
!pip3 install -q transformers==4.5.1

##Chuẩn bị dữ liệu

In [ ]:
import transformers
import torch
import pytorch_lightning as pl

In [ ]:
import pandas as pd
import numpy as np
df['aspect'] = 'None'
xtrain = pd.read_csv('/content/drive/MyDrive/dataclean/dataclean.csv')
for i in range(len(df)):
  cols = []
  dem=0
  for col in df.columns:
    if df[col][i]==1:
      cols+=[col]  
      dem+=1  
  if dem==0:
    df = df.drop(index=i)
    xtrain = xtrain.drop(index=i)
  else:
    df['aspect'][i]=cols
xtrain = df['comment']
labels = df["aspect"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
xtrain['labels'] = labels

In [ ]:
xtrain.head(10)

,comment,labels
0,rất tuyệt vời khi bạn mất điện thoại vẫn không...,[backup_positive]
1,rất thích khả năng lưu trữ nhiều ảnh và chức n...,"[feature_positive, backup_positive]"
2,tôi không hiểu tại sao bao nhiêu năm tôi dùng ...,[stability_negative]
3,sao lưu nhanh miễn phí tạo ảnh hoạt cảnh pa...,"[edit_positive, backup_positive]"
4,gogle đệ tam tuyệt đẹp hoàn hảo rất có í...,[general_positive]
5,tải về rồi mở không lên lưu hình giờ không xe...,"[feature_negative, stability_negative]"
6,quyết định chuyển từ flickr sang thấy cái này...,[general_positive]
7,tôi yêu phần mềm này nếu mất gmail tôi sẽ khó...,[general_positive]
8,rất thích và trên cả tuyệt vời cám ơn gogle,[general_positive]
9,cặp nhan xg không tu dong sao lu ma no tu dong...,"[feature_negative, backup_negative, update_neg..."


##Chia dữ liệu
 

In [ ]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(xtrain,labels,test_size=0.1,train_size=0.9,random_state=0)
x_train, x_dev, y_train, y_dev = train_test_split(x,y,test_size = 0.25,train_size =0.75,random_state=0)

In [ ]:
x_train = x_train.reset_index()['comment']
x_dev = x_dev.reset_index()['comment']
x_test = x_test.reset_index()['comment']

In [ ]:
x_train

0       nếu đồng bộ ảnh được sắp xếp theo như thư mục ...
1        cám ơn  gogle đã tạo ra 1 ứng dụng tuyệt vời ...
2       nên có chế độ xem ảnh backup theo thư mục  để ...
3         chưa được hoàn thiện lắm cần thiết thực hơn nữa
4       méo cha   cập nhật bọn này lại ảnh và video hi...
                              ...                        
3302    ứng dụng hữu ích giúp bộ nhớ máy đt của tôi kh...
3303    tự động xoá đi các album ảnh đã lưu  thất đáng...
3304    ứng dụng rất tốt nhưng ad hạn chế bớt cập nhật...
3305    k vào  được     mong nhà sản xuất mau chóng kh...
3306    ứng dụng rất tuyệt vời  hữu ích  dễ sử d...
Name: comment, Length: 3307, dtype: object

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer



mlb = MultiLabelBinarizer()
mlb.fit(y_train)

y_train = mlb.transform(y_train)
y_dev = mlb.transform(y_dev)
y_test = mlb.transform(y_test)

In [ ]:
print(y_train[-2])
print(mlb.inverse_transform(y_train[-2].reshape(1,-1)))
print(mlb.classes_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[('stability_negative',)]
['backup_negative' 'backup_neutral' 'backup_positive' 'display_negative'
 'display_neutral' 'display_positive' 'edit_negative' 'edit_neutral'
 'edit_positive' 'feature_negative' 'feature_neutral' 'feature_positive'
 'general_negative' 'general_neutral' 'general_positive' 'other_negative'
 'other_neutral' 'other_positive' 'stability_negative' 'stability_neutral'
 'stability_positive' 'update_negative' 'update_neutral' 'update_positive']


##Tách từ

In [ ]:
!pip3 install -q vncorenlp

import os
from vncorenlp import VnCoreNLP
from nltk import flatten

rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/vncorenlp/VnCoreNLP-1.1.1.jar", 
                         annotators="wseg", max_heap_size='-Xmx500m') 

def tokenize(sentence):
        return ' '.join(i for i in flatten(rdrsegmenter.tokenize(sentence)))

print("[INFO] Tokenizing...")
x_train = x_train.apply(tokenize)
print("[INFO] Tokenizing...")
x_dev = x_dev.apply(tokenize)
print("[INFO] Tokenizing...")
x_test = x_test.apply(tokenize)

[INFO] Tokenizing...
[INFO] Tokenizing...
[INFO] Tokenizing...


##Thiết lập mô hình

In [ ]:
BERT_MODEL_NAME = 'vinai/phobert-base'
#BERT_MODEL_NAME = 'bert-base-multilingual-uncased'
#BERT_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = transformers.AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
'''import torch
from transformers import BertTokenizer,BertModel
tokenizer= BertTokenizer.from_pretrained("NlpHUST/vibert4news-base-cased")
bert_model = BertModel.from_pretrained("NlpHUST/vibert4news-base-cased")
BERT_MODEL_NAME = "NlpHUST/vibert4news-base-cased"'''

'import torch\nfrom transformers import BertTokenizer,BertModel\ntokenizer= BertTokenizer.from_pretrained("NlpHUST/vibert4news-base-cased")\nbert_model = BertModel.from_pretrained("NlpHUST/vibert4news-base-cased")\nBERT_MODEL_NAME = "NlpHUST/vibert4news-base-cased"'

##Chuẩn bị Dataset cho Pytorch

In [ ]:
#Build dataset for ABSA pytorch
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, text, labels, tokenizer, max_token_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = labels
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # input_ids = inputs['input_ids'].flatten()
        # attn_mask = inputs['attention_mask'].flatten()
        # token_type_ids = inputs["token_type_ids"]

        return dict(
            text=text,
            input_ids=inputs['input_ids'].flatten(),
            attention_mask=inputs['attention_mask'].flatten(),
            labels=torch.tensor(self.labels[item_idx], dtype=torch.float)
        )

##Thiết lập thông số mô hình

In [ ]:
MAX_LEN = 140
N_EPOCHS = 5
BATCH_SIZE = 8

In [ ]:
train_dataset = BERTDataset(
    x_train,
    y_train,
    tokenizer=tokenizer,
    max_token_len=MAX_LEN
)

sample_item = train_dataset[2458]
sample_item.keys()

dict_keys(['text', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
print(sample_item["text"])
print(sample_item["labels"])
print(sample_item["input_ids"].shape)

cam on gogle da tao dieu kien tốt nhat de minh duoc lien ket voi be ban than quen
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
torch.Size([140])


In [ ]:
bert_model = transformers.AutoModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
sample_batch = next(iter(torch.utils.data.DataLoader(train_dataset, batch_size=8, num_workers=2)))
sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

(torch.Size([8, 140]), torch.Size([8, 140]))

In [ ]:
output = bert_model(sample_batch["input_ids"], sample_batch["attention_mask"])
output.last_hidden_state.shape, output.pooler_output.shape

(torch.Size([8, 140, 768]), torch.Size([8, 768]))

In [ ]:
bert_model.config.hidden_size

768

##Chuẩn bị DataModule


In [ ]:
class BERTDataModule(pl.LightningDataModule):

    def __init__(self, x_train, y_train, x_dev, y_dev, x_test, y_test, tokenizer, batch_size=16, max_token_len=200):
        super().__init__()
        self.train_text = x_train
        self.train_label = y_train
        self.val_text = x_dev
        self.val_label = y_dev
        self.test_text = x_test
        self.test_label = y_test
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self):
        self.train_dataset = BERTDataset(
            text=self.train_text, labels=self.train_label, tokenizer=self.tokenizer, max_token_len=self.max_token_len)
        self.val_dataset = BERTDataset(
            text=self.val_text, labels=self.val_label, tokenizer=self.tokenizer, max_token_len=self.max_token_len)
        self.test_dataset = BERTDataset(
            text=self.test_text, labels=self.test_label, tokenizer=self.tokenizer, max_token_len=self.max_token_len)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2)

In [ ]:
BERTdata_module = BERTDataModule(
    x_train, y_train, x_dev, y_dev, x_test, y_test, tokenizer, BATCH_SIZE, MAX_LEN)
BERTdata_module.setup()

Thiết lập mô hình


In [ ]:
class BERTClassifier(pl.LightningModule):

    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained(
            BERT_MODEL_NAME, return_dict=True)
        self.classifier = torch.nn.Linear(
            self.bert.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = torch.nn.BCELoss()

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.sigmoid(output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
        return loss, output

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_epoch_end(self, outputs):

        labels = []
        predictions = []
        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)

        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)

        for i, name in enumerate(mlb.classes_):
            class_roc_auc = pl.metrics.functional.auroc(
                predictions[:, i], labels[:, i], pos_label=1)
            self.logger.experiment.add_scalar(
                f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

    def configure_optimizers(self):

        optimizer = transformers.AdamW(self.parameters(), lr=2e-5)

        scheduler = transformers.get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.n_warmup_steps,
            num_training_steps=self.n_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [ ]:
steps_per_epoch = len(x_train) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [ ]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(413, 2065)

In [ ]:
model = BERTClassifier(
    n_classes=len(mlb.classes_),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
)

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filename="PhoBERT-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = pl.loggers.TensorBoardLogger("lightning_logs", name="ABSA")
early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss', patience=2)

##Huấn luyện mô hình

In [ ]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, BERTdata_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | bert       | RobertaModel | 134 M 
1 | classifier | Linear       | 18.5 K
2 | criterion  | BCELoss      | 0     
--------------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
540.067   Total estimated model params size (MB)


Epoch 0, global step 413: val_loss reached 0.19901 (best 0.19901), saving model to "lightning_logs/ABSA/version_0/checkpoints/PhoBERT-epoch=00-val_loss=0.20.ckpt" as top 1


Epoch 1, global step 827: val_loss reached 0.16506 (best 0.16506), saving model to "lightning_logs/ABSA/version_0/checkpoints/PhoBERT-epoch=01-val_loss=0.17.ckpt" as top 1


Epoch 2, global step 1241: val_loss reached 0.14790 (best 0.14790), saving model to "lightning_logs/ABSA/version_0/checkpoints/PhoBERT-epoch=02-val_loss=0.15.ckpt" as top 1


Epoch 3, global step 1655: val_loss reached 0.14048 (best 0.14048), saving model to "lightning_logs/ABSA/version_0/checkpoints/PhoBERT-epoch=03-val_loss=0.14.ckpt" as top 1


Epoch 4, global step 2069: val_loss reached 0.13661 (best 0.13661), saving model to "lightning_logs/ABSA/version_0/checkpoints/PhoBERT-epoch=04-val_loss=0.14.ckpt" as top 1


1

In [ ]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.13136547803878784}
--------------------------------------------------------------------------------


[{'test_loss': 0.13136547803878784}]

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir lightning_logs/

In [ ]:
trained_model = BERTClassifier.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path,
    n_classes=len(mlb.classes_)
)
trained_model.eval()
trained_model.freeze()

##Đánh giá mô hình

In [ ]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

val_dataset = BERTDataset(
    x_test,
    y_test,
    tokenizer,
    MAX_LEN
)

predictions = []
labels = []

for i in tqdm(range(len(val_dataset))):
    _, prediction = trained_model(
        val_dataset[i]["input_ids"].unsqueeze(dim=0).to(device),
        val_dataset[i]["attention_mask"].unsqueeze(dim=0).to(device)
    )
    predictions.append(prediction.flatten())
    labels.append(val_dataset[i]["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [ ]:
pl.metrics.functional.accuracy(predictions, labels, threshold=0.5)

tensor(0.9549)

In [ ]:
pl.metrics.functional.f1(predictions, labels, len(
    mlb.classes_), 0.5, 'micro', True)

tensor(0.5569)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(classification_report(
    y_true,
    y_pred,
    target_names=mlb.classes_,
    zero_division=0
))

                    precision    recall  f1-score   support

   backup_negative       0.86      0.08      0.15        72
    backup_neutral       0.00      0.00      0.00        11
   backup_positive       0.87      0.89      0.88       152
  display_negative       0.00      0.00      0.00         7
   display_neutral       0.00      0.00      0.00         4
  display_positive       0.00      0.00      0.00         7
     edit_negative       0.00      0.00      0.00         6
      edit_neutral       0.00      0.00      0.00         5
     edit_positive       0.00      0.00      0.00        13
  feature_negative       0.00      0.00      0.00        43
   feature_neutral       0.00      0.00      0.00        46
  feature_positive       0.00      0.00      0.00        35
  general_negative       0.00      0.00      0.00        17
   general_neutral       0.00      0.00      0.00         6
  general_positive       0.77      0.76      0.77       187
    other_negative       0.00      0.00

In [ ]:
print("Precision_score: ",precision_score(y_true,y_pred, average='micro'))
print("recall_score:",recall_score(y_true,y_pred, average='micro'))
print("f1_score:",f1_score(y_true,y_pred, average='micro'))

Precision_score:  0.7672811059907834
recall_score: 0.43700787401574803
f1_score: 0.5568561872909699


In [ ]:
pred = mlb.inverse_transform(np.array(y_pred))
act = mlb.inverse_transform(y_true)

df = pd.DataFrame({"Body": x_test, "Actual": act, "Predicted": pred})
df.head(10)

,Body,Actual,Predicted
0,k có phần_mềm chỉnh ảnh như phiên_bản cũ sao l...,"(backup_negative, feature_negative, update_neg...","(stability_negative,)"
1,ap dùng tạm ổn nhưng mỗi lần tôi lưu ảnh thì v...,"(backup_negative, general_neutral, stability_n...",()
2,không có ngăn bảo_mật và chỉnh_sửa video hình_ảnh,"(edit_negative, feature_negative)",()
3,trợ_lý quá đỉnh suguest mấy hiệu_ứng với ghép ...,"(feature_positive,)","(general_positive,)"
4,very tốt 5star ok team gogle lc ứng_dụng rất v...,"(backup_positive, general_positive)","(backup_positive, general_positive)"
5,gogle photos giúp mình lưu_trữ tất_cả hình_ảnh...,"(backup_positive, general_positive)","(backup_positive,)"
6,thiet lap ung dumg moi that tuyet voi muot ma ...,"(feature_positive,)","(general_positive,)"
7,ứng_dụng kì_cục vậy cập_nhật hoài mà cập_nhật ...,"(stability_negative, update_negative)","(stability_negative,)"
8,ứng_dụng rất tuyệt chế_độ tự gép ảnh rất thông...,"(backup_positive, feature_positive, general_po...","(general_positive,)"
9,tuyệt_vời có_thể lưu được nhiều ảnh mà không t...,"(backup_positive, general_positive)","(backup_positive, general_positive)"
